In [1]:
import pandas as pd
import sqlite3 
import numpy as np
import random

In [5]:
conn = sqlite3.connect("stocks.db")

In [6]:
df = pd.read_sql_query("SELECT * FROM stocks", conn)

## Random selection of stocks

In [8]:
max_price = 20    
## Selecting tickers only with price lower than max_price from most recent date
selected_tickers = df.loc[(df["date"] == df["date"].max()) & (df["adjusted"] <= max_price), "adjusted"]

## Getting data from n-previous dates for selected tickers 
n_recent = 30
recent_dates = df["date"].unique()[-n_recent:]

## Selecting df data
df_selected = df.loc[(df["date"].isin(recent_dates)) & (df["ticker"].isin(selected_tickers))]

## Leaving only needed attributes. Ale to chyba trzeba wyjebać wcześniej
df_selected = df_selected.loc[:,["ticker", "date", "adjusted"]]

In [25]:
def sr_selector(df, max_price, ndays = 30, number_of_stocks_in_portfolio = 4, n_probes = 100, sr_iterations = 100):
    
    ## Selecting tickers only with price lower than max_price from most recent date
    selected_tickers = df.loc[(df["date"] == df["date"].max()) & (df["adjusted"] <= max_price), "ticker"]
    
    ## Getting data from n-previous dates for selected tickers 
    
    recent_dates = df["date"].unique()[-ndays:]
    

    ## Selecting df data
    df_selected = df.loc[(df["date"].isin(recent_dates)) & (df["ticker"].isin(selected_tickers))]

    ## Leaving only needed attributes. Ale to chyba trzeba wyjebać wcześniej
    df_selected = df_selected.loc[:,["ticker", "date", "adjusted"]]
    
    
    ## Defining sharpe results list to store results from iteration
    
    sharpe_results = []
    
    ######################################## Actual sr selection ##############################################
    
    ## How many times we want to draw random combination of tickers to find best sharpe portfolio
    for nprobe in range(n_probes):
        
        unique_random = False

        ## Drawing tickers will continue till the list of tickers consists only of unique tickers
        while unique_random == False:
            
            ## Drawing tickers randomly
            random_tickers = np.random.choice(df_selected["ticker"], number_of_stocks_in_portfolio)
            random_tickers = list(random_tickers)


            if len(list(set(random_tickers))) == number_of_stocks_in_portfolio:
                unique_random = True

        ## Data for random tickers
        df_random = df_selected.loc[df_selected["ticker"].isin(random_tickers)]

        ## Transposing values from rows to column to get separate columns for each ticker with Adj Close values
        df_random = df_random.pivot(index = 'date', columns = "ticker", values="adjusted")


        ## Generating log return for stocks selected
        log_rets = np.log(df_random/df_random.shift(1))

        SR_final = -10000
        weights_final = ""


        for i in range(sr_iterations):

            ## Drawing random weights 
            weights = np.array(np.random.random(number_of_stocks_in_portfolio))

            ## Rebalancing random weights (so they sum up to 1)
            weights = weights/weights.sum()

            ## Expected returns
            exp_ret = np.sum((log_rets.mean() * weights) * 252)

            ## Expected volatility 
            exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_rets.cov() * 252, weights)))

            ## Sharpe Ratio
            SR = exp_ret/exp_vol

            if SR > SR_final:

                SR_final = SR
                weights_final = weights.copy()
        
        sharpe_results.append((str(random_tickers), SR_final, str(weights_final)))
        
    df_sharpe_results = pd.DataFrame(sharpe_results, columns = ['tickers', 'sharpe_ratio', 'weights'])
    df_sharpe_results.sort_values('sharpe_ratio', ascending = False, inplace = True)
    return df_sharpe_results

In [26]:
sharpe = sr_selector(df, 30)

In [27]:
sharpe

,tickers,sharpe_ratio,weights
28,"['OLN', 'ACB', 'LEVI', 'SKT']",7.130998,[0.01320394 0.37922576 0.5196537 0.0879166 ]
71,"['DAN', 'TRGP', 'NOK', 'INFY']",6.325198,[0.04721091 0.914488 0.00773908 0.03056201]
5,"['GNTX', 'LBTYK', 'ORBC', 'OLN']",5.739975,[0.07465293 0.06195372 0.8269116 0.03648175]
72,"['XXII', 'KMI', 'SUM', 'AIG']",5.095385,[0.1236231 0.05532594 0.3445919 0.47645906]
62,"['XRX', 'UA', 'SU', 'ANF']",4.426491,[0.69339619 0.05194103 0.227903 0.02675977]
39,"['PS', 'ANF', 'TME', 'QD']",4.325204,[0.75271113 0.20836537 0.01136107 0.02756244]
70,"['TV', 'ANF', 'SKM', 'IRM']",4.243190,[0.59491931 0.01095093 0.24200008 0.15212968]
93,"['NLY', 'CLMT', 'TEO', 'CAR']",4.178804,[0.01480845 0.68023511 0.05397371 0.25098273]
87,"['FCX', 'ENB', 'XXII', 'CORE']",4.089630,[0.00227519 0.18806673 0.31043283 0.49922525]
14,"['AMX', 'NLY', 'ALLY', 'RST']",3.535063,[0.25828294 0.05493339 0.05112639 0.63565728]


In [9]:
def random_selector(number_of_random_tickers, sr_iterations):
    
    
    unique_random = False
    
    while unique_random == False:
        
        random_tickers = np.random.choice(df_selected["Ticker"], number_of_random_tickers)
        random_tickers = list(random_tickers)
        
              
        if len(list(set(random_tickers))) == number_of_random_tickers:
            unique_random = True
    
    
    ## Data for random tickers
    df_random = df_selected.loc[df_selected["Ticker"].isin(random_tickers)]
    
    ## Transposing values from rows to column to get separate columns for each ticker with Adj Close values
    df_random = df_random.pivot(index = 'Date', columns = "Ticker", values="Adj Close")
    
    
    ## Generating log return for stocks selected
    log_rets = np.log(df_random/df_random.shift(1))

    SR_final = -10000
    weights_final = ""


    for i in range(sr_iterations):

        ## Drawing random weights 
        weights = np.array(np.random.random(number_of_random_tickers))

        ## Rebalancing random weights (so they sum up to 1)
        weights = weights/weights.sum()

        ## Expected returns
        exp_ret = np.sum((log_rets.mean() * weights) * 252)

        ## Expected volatility 
        exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_rets.cov() * 252, weights)))

        ## Sharpe Ratio
        SR = exp_ret/exp_vol

        if SR > SR_final:

            SR_final = SR
            weights_final = weights.copy()


    to_return = (random_tickers, SR_final, weights_final)
    return to_return

## SR for one stock

In [6]:
max_price = 20    
## Selecting tickers only with price lower than max_price from most recent date
selected_tickers = df.loc[(df["Date"] == df["Date"].max()) & (df["Adj Close"] <= max_price), "Ticker"]

## Getting data from n-previous dates for selected tickers 
n_recent = 7
recent_dates = df["Date"].unique()[-n_recent:]

## Selecting df data
df_selected = df.loc[(df["Date"].isin(recent_dates)) & (df["Ticker"].isin(selected_tickers))]

## Leaving only needed attributes. Ale to chyba trzeba wyjebać wcześniej
df_selected = df_selected.loc[:,["Ticker", "Date", "Adj Close"]]

df_selected = df_selected.pivot(index =  'Date', columns = 'Ticker', values = 'Adj Close')

In [7]:
def sharpe_ratio(data_for_one_ticker):
    
    ## Generating log return for stocks selected
    log_rets = np.log(data_for_one_ticker/data_for_one_ticker.shift(1))
    
    ## Expected returns
    exp_ret = log_rets.mean()

    ## Expected volatility 
    exp_vol = log_rets.std()

    ## Sharpe Ratio
    SR = exp_ret/exp_vol * (252) **0.5
    
    return SR

In [8]:
scores = df_selected.apply(sharpe_ratio, axis = 0)

In [9]:
pd.DataFrame(scores).sort_values(0, ascending=False)

,0
Ticker,
SWN,9.082176
RENN,8.471791
QIWI,7.745285
GPRO,5.841505
OMI,4.670404
UNFI,4.113079
BBBY,3.814779
QFIN,3.160082
MOS,2.338748
